In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip uninstall nltk -y
!pip install texthero

Uninstalling nltk-3.2.5:
  Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 245kB 7.7MB/s 
     |████████████████████████████████| 1.4MB 9.8MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp37-none-any.whl size=1434674 sha256=f44ca9043c468acb47a75136a4a893f22518fcfdd855c0b480b39ee5ce63ab43
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk


In [3]:
import os

import numpy as np
import pandas as pd

import dill
from sklearn.feature_extraction.text import TfidfVectorizer
import gc
from tqdm import tqdm
import time
import nltk

import texthero as hero
from texthero import preprocessing

INPUT_DIR = '/content/drive/MyDrive/citation_prediction/input/'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


データの読み込み

In [4]:
all_title_abstract = pd.read_feather(os.path.join(INPUT_DIR, 'all_title_abstract_df.feather'))
all_title_abstract.head()

,title,abstract,cites,clean_title,clean_abstract
0,Heavy-Flavour Production at HERA,We review the theoretical and experimental s...,NaN,heavy flavour production hera,review theoretical experimental status hea...
1,Spectropolarimetric Constraints on the Nature ...,While it is well recognized that interstella...,7.0,spectropolarimetric constraints nature inte...,well recognized interstellar grains made ...
2,A joint analysis of Planck and BICEP2 B modes ...,We analyze BICEP2 and Planck data using a mo...,188.0,joint analysis planck bicep2 b modes includ...,analyze bicep2 planck data using model inc...
3,Molecular movie of ultrafast coherent rotation...,Recording molecular movies on ultrafast time...,8.0,molecular movie ultrafast coherent rotational...,recording molecular movies ultrafast timescal...
4,A Modified Mixed Domain Method for Modeling Ac...,"In this paper, phase correction and amplitud...",NaN,modified mixed domain method modeling acoust...,paper phase correction amplitude compensati...


In [5]:
file = os.path.join(INPUT_DIR, 'fasttext_model_title.dill')
title_fasttext_model = dill.load(open(file,'rb'))

In [6]:
# ベクトル化
tfidf_vectorizer = TfidfVectorizer(min_df=20, lowercase=False, token_pattern="(?u)\\b\\w+\\b")
tfidf_vectorizer.fit(all_title_abstract['clean_title'].values)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=False, max_df=1.0, max_features=None,
                min_df=20, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [7]:
idf_dic = dict(zip(tfidf_vectorizer.get_feature_names(), tfidf_vectorizer._tfidf.idf_))

In [8]:
for word in title_fasttext_model.wv.index2word:
    if not word in idf_dic.keys():
        idf_dic[word] = 0

In [9]:
tqdm.pandas()

def des_to_mean_vec(text):
    
    try:
        text = text.strip()
        text_ls = [s for s in text.split(' ') if '' != s]
        return np.mean([title_fasttext_model.wv[word] * idf_dic[word] for word in text_ls if word in title_fasttext_model.wv.index2word], axis=0)
    except:
        return np.nan

df = all_title_abstract['clean_title'].progress_apply(lambda x: des_to_mean_vec(x))
df.head()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning:

The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version

  0%|          | 877/910608 [00:00<08:57, 1691.36it/s]/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning:

Mean of empty slice.

100%|██████████| 910608/910608 [08:19<00:00, 1822.85it/s]


0    [0.14014913, -0.054490075, 0.20272276, 0.62840...
1    [0.013459873, 0.0026483715, 0.069263145, 0.483...
2    [-0.051640946, -0.29469207, -0.41444278, 0.362...
3    [-0.18325152, -0.09797374, 0.088996015, 0.8019...
4    [-0.010437534, -0.35722283, -0.041812, 0.41457...
Name: clean_title, dtype: object

In [10]:
title_df = pd.DataFrame()
for num in tqdm(list(range(0, 920000, 10000))):
    title_df = pd.concat([title_df, 
                          df.iloc[num:num+10000].apply(pd.Series)])
title_df.columns = ['fasttext_tfidf_title_' + str(col) for col in title_df.columns]
title_df.head()

100%|██████████| 92/92 [03:07<00:00,  2.04s/it]


,fasttext_tfidf_title_0,fasttext_tfidf_title_1,fasttext_tfidf_title_2,fasttext_tfidf_title_3,fasttext_tfidf_title_4,fasttext_tfidf_title_5,fasttext_tfidf_title_6,fasttext_tfidf_title_7,fasttext_tfidf_title_8,fasttext_tfidf_title_9,fasttext_tfidf_title_10,fasttext_tfidf_title_11,fasttext_tfidf_title_12,fasttext_tfidf_title_13,fasttext_tfidf_title_14,fasttext_tfidf_title_15,fasttext_tfidf_title_16,fasttext_tfidf_title_17,fasttext_tfidf_title_18,fasttext_tfidf_title_19,fasttext_tfidf_title_20,fasttext_tfidf_title_21,fasttext_tfidf_title_22,fasttext_tfidf_title_23,fasttext_tfidf_title_24,fasttext_tfidf_title_25,fasttext_tfidf_title_26,fasttext_tfidf_title_27,fasttext_tfidf_title_28,fasttext_tfidf_title_29,fasttext_tfidf_title_30,fasttext_tfidf_title_31,fasttext_tfidf_title_32,fasttext_tfidf_title_33,fasttext_tfidf_title_34,fasttext_tfidf_title_35,fasttext_tfidf_title_36,fasttext_tfidf_title_37,fasttext_tfidf_title_38,fasttext_tfidf_title_39,...,fasttext_tfidf_title_160,fasttext_tfidf_title_161,fasttext_tfidf_title_162,fasttext_tfidf_title_163,fasttext_tfidf_title_164,fasttext_tfidf_title_165,fasttext_tfidf_title_166,fasttext_tfidf_title_167,fasttext_tfidf_title_168,fasttext_tfidf_title_169,fasttext_tfidf_title_170,fasttext_tfidf_title_171,fasttext_tfidf_title_172,fasttext_tfidf_title_173,fasttext_tfidf_title_174,fasttext_tfidf_title_175,fasttext_tfidf_title_176,fasttext_tfidf_title_177,fasttext_tfidf_title_178,fasttext_tfidf_title_179,fasttext_tfidf_title_180,fasttext_tfidf_title_181,fasttext_tfidf_title_182,fasttext_tfidf_title_183,fasttext_tfidf_title_184,fasttext_tfidf_title_185,fasttext_tfidf_title_186,fasttext_tfidf_title_187,fasttext_tfidf_title_188,fasttext_tfidf_title_189,fasttext_tfidf_title_190,fasttext_tfidf_title_191,fasttext_tfidf_title_192,fasttext_tfidf_title_193,fasttext_tfidf_title_194,fasttext_tfidf_title_195,fasttext_tfidf_title_196,fasttext_tfidf_title_197,fasttext_tfidf_title_198,fasttext_tfidf_title_199
0,0.140149,-0.054490,0.202723,0.628401,-0.014993,-0.003029,0.093187,0.053646,0.155185,0.281145,0.336346,-0.423085,0.439124,0.278963,0.192753,-0.241766,-0.317880,0.390388,0.025266,0.257371,-0.502624,0.318292,0.423081,0.420846,-0.426469,-0.360286,0.773780,0.358386,0.668082,0.126170,-0.269790,0.054916,0.368953,0.246326,0.142240,-0.232998,0.023021,0.152774,-0.147791,-0.008986,...,0.356388,0.135711,-0.130194,-0.709340,0.382379,0.142266,0.335764,0.411784,0.196323,0.299895,-0.285681,-0.052241,-0.291273,0.399589,0.215108,-0.234766,-0.164668,0.245501,0.311385,-0.259384,0.023057,0.451768,0.239780,0.116364,-0.209816,-0.210390,0.152256,-0.148024,0.398591,0.192228,0.079427,-0.166951,0.557345,-0.424619,-0.155822,-0.040717,0.596780,0.212754,-0.050893,-0.110402
1,0.013460,0.002648,0.069263,0.483768,-0.720808,-0.604486,0.240943,0.533478,-0.312485,0.091908,0.387502,0.129819,0.226491,0.758366,-0.436525,-0.099985,-0.165403,0.168166,-0.336487,0.059216,-0.377721,-0.232376,0.468632,0.032783,-0.676661,-0.006740,-0.012259,0.425269,-0.313182,-0.172623,0.253804,-0.093091,-0.271467,0.625414,0.080171,0.057466,0.749135,-0.157838,0.027179,-0.233605,...,0.071820,-0.090397,-0.112490,-0.681452,0.056845,-0.046542,0.178516,0.382902,0.018294,-0.284063,-0.229110,0.553474,0.033392,0.230126,0.373552,-0.186387,0.394596,-0.199563,0.331765,-0.126082,-0.294180,0.194424,0.092723,-0.114917,0.413837,-0.098325,-0.222891,-0.051317,-0.308127,0.105346,-0.335278,-0.352885,0.417923,-0.281469,-0.311932,-0.284983,-0.325586,-0.103496,-0.469040,0.760347
2,-0.051641,-0.294692,-0.414443,0.362708,-0.251421,-0.225286,0.287944,0.165095,-0.084777,0.149528,-0.056010,0.093904,0.134428,0.204384,-0.179891,-0.446817,-0.289074,0.360723,-0.581133,0.158366,-0.481777,0.226365,0.191431,0.080205,-0.618319,-0.348511,0.279168,0.118157,-0.162938,-0.306260,-0.008141,0.275040,0.511249,0.121996,-0.275293,-0.263796,0.179073,0.364369,0.239964,-0.283082,...,0.625332,-0.147701,-0.069428,-0.376843,0.088883,-0.079966,0.039873,0.059544,-0.234059,-0.203838,0.177488,0.604595,-0.443093,0.052436,0.3

In [11]:
title_df.to_feather(os.path.join(INPUT_DIR, 'title_fasttext_tfidf_df.feather'))

del title_df
gc.collect()

0

In [12]:
file = os.path.join(INPUT_DIR, 'fasttext_model_abstract.dill')
abstract_fasttext_model = dill.load(open(file,'rb'))

In [13]:
# ベクトル化
tfidf_vectorizer = TfidfVectorizer(min_df=20, lowercase=False, token_pattern="(?u)\\b\\w+\\b")
tfidf_vectorizer.fit(all_title_abstract['clean_abstract'].values)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=False, max_df=1.0, max_features=None,
                min_df=20, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [14]:
idf_dic = dict(zip(tfidf_vectorizer.get_feature_names(), tfidf_vectorizer._tfidf.idf_))

In [15]:
for word in abstract_fasttext_model.wv.index2word:
    if not word in idf_dic.keys():
        idf_dic[word] = 0

In [16]:
tqdm.pandas()

def des_to_mean_vec(text):
    
    try:
        text = text.strip()
        text_ls = [s for s in text.split(' ') if '' != s]
        return np.mean([abstract_fasttext_model.wv[word] * idf_dic[word] for word in text_ls if word in abstract_fasttext_model.wv.index2word], axis=0)
    except:
        return np.nan

df = all_title_abstract['clean_abstract'].progress_apply(lambda x: des_to_mean_vec(x))
df.head()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning:

The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version

  4%|▎         | 31980/910608 [06:30<2:59:01, 81.80it/s]/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning:

Mean of empty slice.

100%|██████████| 910608/910608 [3:00:03<00:00, 84.29it/s]


0    [0.29557112, -0.51116616, 0.023048945, 0.03107...
1    [0.52664113, -0.32263228, -0.08731426, -0.1583...
2    [0.2039427, -0.29791096, 0.04954452, -0.045631...
3    [0.18584576, -0.32431182, 0.011944047, -0.0714...
4    [0.2568219, -0.23003556, 0.023119055, -0.12335...
Name: clean_abstract, dtype: object

In [17]:
abstract_df = pd.DataFrame()
for num in tqdm(list(range(0, 920000, 10000))):
    abstract_df = pd.concat([abstract_df, 
                             df.iloc[num:num+10000].apply(pd.Series)])
# abstract_df = df.apply(pd.Series)
abstract_df.columns = ['fasttext_tfidf_abstract_' + str(col) for col in abstract_df.columns]
abstract_df.head()

100%|██████████| 92/92 [03:21<00:00,  2.19s/it]


,fasttext_tfidf_abstract_0,fasttext_tfidf_abstract_1,fasttext_tfidf_abstract_2,fasttext_tfidf_abstract_3,fasttext_tfidf_abstract_4,fasttext_tfidf_abstract_5,fasttext_tfidf_abstract_6,fasttext_tfidf_abstract_7,fasttext_tfidf_abstract_8,fasttext_tfidf_abstract_9,fasttext_tfidf_abstract_10,fasttext_tfidf_abstract_11,fasttext_tfidf_abstract_12,fasttext_tfidf_abstract_13,fasttext_tfidf_abstract_14,fasttext_tfidf_abstract_15,fasttext_tfidf_abstract_16,fasttext_tfidf_abstract_17,fasttext_tfidf_abstract_18,fasttext_tfidf_abstract_19,fasttext_tfidf_abstract_20,fasttext_tfidf_abstract_21,fasttext_tfidf_abstract_22,fasttext_tfidf_abstract_23,fasttext_tfidf_abstract_24,fasttext_tfidf_abstract_25,fasttext_tfidf_abstract_26,fasttext_tfidf_abstract_27,fasttext_tfidf_abstract_28,fasttext_tfidf_abstract_29,fasttext_tfidf_abstract_30,fasttext_tfidf_abstract_31,fasttext_tfidf_abstract_32,fasttext_tfidf_abstract_33,fasttext_tfidf_abstract_34,fasttext_tfidf_abstract_35,fasttext_tfidf_abstract_36,fasttext_tfidf_abstract_37,fasttext_tfidf_abstract_38,fasttext_tfidf_abstract_39,...,fasttext_tfidf_abstract_160,fasttext_tfidf_abstract_161,fasttext_tfidf_abstract_162,fasttext_tfidf_abstract_163,fasttext_tfidf_abstract_164,fasttext_tfidf_abstract_165,fasttext_tfidf_abstract_166,fasttext_tfidf_abstract_167,fasttext_tfidf_abstract_168,fasttext_tfidf_abstract_169,fasttext_tfidf_abstract_170,fasttext_tfidf_abstract_171,fasttext_tfidf_abstract_172,fasttext_tfidf_abstract_173,fasttext_tfidf_abstract_174,fasttext_tfidf_abstract_175,fasttext_tfidf_abstract_176,fasttext_tfidf_abstract_177,fasttext_tfidf_abstract_178,fasttext_tfidf_abstract_179,fasttext_tfidf_abstract_180,fasttext_tfidf_abstract_181,fasttext_tfidf_abstract_182,fasttext_tfidf_abstract_183,fasttext_tfidf_abstract_184,fasttext_tfidf_abstract_185,fasttext_tfidf_abstract_186,fasttext_tfidf_abstract_187,fasttext_tfidf_abstract_188,fasttext_tfidf_abstract_189,fasttext_tfidf_abstract_190,fasttext_tfidf_abstract_191,fasttext_tfidf_abstract_192,fasttext_tfidf_abstract_193,fasttext_tfidf_abstract_194,fasttext_tfidf_abstract_195,fasttext_tfidf_abstract_196,fasttext_tfidf_abstract_197,fasttext_tfidf_abstract_198,fasttext_tfidf_abstract_199
0,0.295571,-0.511166,0.023049,0.031080,-0.117655,-0.297365,-0.149563,0.602817,0.136406,-0.084433,-0.097949,0.121233,0.093626,-0.272161,0.459869,-0.316856,-0.087571,0.362089,-0.314481,0.215510,0.071674,-0.235536,0.117824,0.005482,-0.380021,0.247584,-0.174011,-0.078768,0.318793,-0.102675,0.180664,-0.254342,0.608679,-0.069173,0.145144,-0.148795,-0.210828,0.160981,-0.180889,-0.499102,...,-0.055008,-0.049391,0.297205,0.129744,-0.607416,-0.294775,0.043979,0.258355,-0.320750,0.106158,-0.056055,-0.007935,-0.266228,0.406964,0.202570,0.119375,-0.059507,-0.055118,0.041769,-0.539250,-0.009630,0.290849,-0.435132,0.078526,0.447569,-0.328798,0.008280,0.195415,0.066167,0.289711,0.010328,-0.471306,-0.235082,0.017867,-0.295492,0.057984,0.040581,0.007417,0.030718,0.376207
1,0.526641,-0.322632,-0.087314,-0.158394,0.231263,-0.040078,-0.114828,0.231837,0.007926,-0.298640,0.212196,-0.169250,-0.226112,-0.070713,0.285715,-0.150709,0.347661,0.169059,0.068398,0.196594,0.097761,-0.186183,-0.131675,-0.125433,-0.218325,0.096663,-0.279845,0.029123,0.108950,-0.245254,0.426017,-0.207758,-0.124485,0.097464,-0.061549,0.207279,0.018005,0.096280,-0.038944,0.149227,...,0.111234,0.217759,0.332332,-0.105064,-0.132720,-0.425614,-0.293244,0.112784,-0.040897,0.430835,-0.173062,-0.245700,-0.160246,0.018052,0.017741,-0.181918,-0.494297,-0.271489,0.429514,-0.058367,0.175783,-0.130877,-0.370556,0.212867,0.145122,0.224508,-0.238760,-0.129128,0.385841,0.172235,0.388096,-0.103977,-0.232018,-0.008779,-0.129145,-0.211067,-0.201892,-0.056464,-0.074197,0.329619
2,0.203943,-0.297911,0.049545,-0.045631,0.178927,-0.022638,-0.192560,0.205874,0.042350,-0.210767,0.037250,-0.257037,-0.249340,-0.093919,0.015936,-0.253247,0.266845,0.092058,0.041858,-0.067802,0.042945,-0.161843,-0.002415,-0.137287,-0.408475,0.020243,-0.204420,0.084133,0.

In [18]:
abstract_df.to_feather(os.path.join(INPUT_DIR, 'abstract_fasttext_tfidf_df.feather'))